# Contribution Analysis Tutorial

This short tutorial shows how to run contribution analysis and generate plots with different settings.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import required functions
import pandas as pd
import bw2data as bd
from mescal.contribution_analysis import process_contribution_data
from mescal.plot import plot_contribution_analysis
from mescal import Database
import sys
import os

In [3]:
# Add parent directory to path to import streamlit_app from root
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from streamlit_app import launch_streamlit_app

2026-02-09 16:09:55.219 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 16:09:55.219 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 16:09:56.201 
  command:

    streamlit run C:\Users\matth\PycharmProjects\mescal\venv\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-02-09 16:09:56.201 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 16:09:56.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 16:09:56.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-09 16:09:56.205 Thread 'MainThread': missing ScriptRunContext! This warning can be

In [4]:
bd.projects.set_current('ecoinvent3.10.1 backup')

## Setup: Define input file and results paths and load data

Set paths to your contribution data, impact scores, and unit conversion files.

In [5]:
# Define paths to input data and results files
results_path = '../examples/results/energyscope_core/2050/'
data_path = '../dev/energyscope_data/core/'

In [6]:
impact_scores_df = pd.read_csv(results_path + 'impact_scores.csv')
contrib_analysis_df = pd.read_csv(results_path + 'contribution_analysis.csv')
unit_conversion_df = pd.read_excel(data_path + 'unit_conversion.xlsx')

Load the technosphere and biosphere databases to get the flows names from uuid.

In [7]:
esm_database_name = "EnergyScope_core_2050"

In [8]:
biosphere_db = Database('biosphere3')
spatialized_biosphere = Database("biosphere3_spatialized_flows")
full_biosphere_db = biosphere_db + spatialized_biosphere
full_biosphere_db_as_dict_code = full_biosphere_db.list_to_dict(key='code', database_type='biosphere')

Getting activity data


100%|██████████| 4362/4362 [00:00<00:00, 182962.86it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


...

100%|██████████| 110559/110559 [00:01<00:00, 93907.37it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 110559/110559 [00:00<00:00, 3470605.82it/s]
2026-02-09 16:10:07,448 - Database - INFO - Loaded biosphere3_spatialized_flows from brightway!


In [9]:
technosphere_db = Database("ecoinvent_cutoff_3.10.1_image_SSP2-L_2050")
technosphere_db_as_dict_code = technosphere_db.list_to_dict(key='code', database_type='technosphere')

2026-02-09 16:10:13,410 - Database - INFO - Loaded ecoinvent_cutoff_3.10.1_image_SSP2-L_2050 from pickle!


In [10]:
contrib_analysis_emissions_df = contrib_analysis_df[contrib_analysis_df['contribution_type'] == 'emissions']
contrib_analysis_processes_df = contrib_analysis_df[contrib_analysis_df['contribution_type'] == 'processes']

In [11]:
contrib_analysis_emissions_df['ef_name'] = contrib_analysis_emissions_df.apply(lambda x: full_biosphere_db_as_dict_code[(x['database'], x['code'])]['name'], axis=1)

In [12]:
contrib_analysis_processes_df['process_name'] = contrib_analysis_processes_df.apply(lambda x: "Direct emissions" if x['database'] == esm_database_name else technosphere_db_as_dict_code[(x['database'], x['code'])]['name'], axis=1)

## 1) Default Settings: Process and Plot

This example shows the basic workflow with default settings. The commented-out export option shows how to export to Excel.


In [13]:
# Process the contribution data with minimal parameters
# - All impacts and activity types will be included
# - Results are NOT exported to Excel by default

contribution_type = "emissions"

grouped_df, unit_type_groups_dict = process_contribution_data(
    contrib_df=contrib_analysis_emissions_df if contribution_type == 'emissions' else contrib_analysis_processes_df,
    impact_scores_df=impact_scores_df,
    unit_conversion_df=unit_conversion_df,
    contribution_type=contribution_type,
    saving_path=results_path,
    export_excel=False,  # Set to True to save results as Excel file
)

In [14]:
# Generate plots for all impact categories with minimal parameters
# Default behavior: includes all data and uses standard styling

# plot_contribution_analysis(
#     df=grouped_df,
#     unit_type_groups_dict=unit_type_groups_dict,
#     output_dir=output_dir,
#     contribution_type='emissions',
# )

## 2) Advanced Example: All Customization Options

This example demonstrates all available settings with inline comments explaining each parameter.


In [15]:
# Process contribution data with all available options
grouped_df_custom, unit_type_groups_dict_custom = process_contribution_data(
    contrib_df=contrib_analysis_emissions_df if contribution_type == 'emissions' else contrib_analysis_processes_df,
    impact_scores_df=impact_scores_df,
    unit_conversion_df=unit_conversion_df,
    contribution_type=contribution_type,
    saving_path=results_path,
    export_excel=False,
)

In [16]:
# Generate customized plots with all available parameters
plot_contribution_analysis(
    df=grouped_df_custom,  # Processed dataframe from process_contribution_data()
    unit_type_groups_dict=unit_type_groups_dict_custom,  # Unit groups dictionary
    contribution_type=contribution_type,  # Type of analysis: 'processes' or 'emissions'
    # Optional filtering parameters:
    act_types=['Operation', 'Construction'],  # Set of activity types to include, e.g., ['Operation', 'Construction'], None for all
    impact_categories_list=["('IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'Midpoint', 'Climate change, long term')"],  # Set to list of strings to filter specific impacts, or None for all
    esm_units=['kilowatt', 'kilowatt hour'],  # Set to list of ESM units to filter, e.g., ['kilowatt'], or None for all
    saving_path=results_path,  # Directory to save plot images
    show_plot=False,  # Set to True to display plots in notebook, False to only save
    # Plot appearance parameters:
    threshold=0.2,  # Minimum contribution share to display (0-1). Lower values show more detail
    cell_size=0.6,  # Height of each cell in the heatmap (in inches)
    min_fig_width=8,  # Minimum figure width in inches
    min_fig_height=5,  # Minimum figure height in inches
    dpi=150,  # Dots per inch for saved plots. Higher = better quality but larger file size
)

C:\Users\matth\PycharmProjects\mescal\mescal\plot.py:870: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_cat = pd.concat([df_cat, Others_share], ignore_index=True)


## 3) Using the Streamlit App

Launch the interactive Streamlit application for a GUI-based workflow with filtering capabilities.


In [ ]:
# Alternative: Launch with both emissions AND processes files for switching between them in the app
launch_streamlit_app(
    contrib_processes_df=contrib_analysis_processes_df,
    contrib_emissions_df=contrib_analysis_emissions_df,
    impact_scores_df=impact_scores_df,
    unit_conversion_df=unit_conversion_df,
)